In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matminer.datasets import load_dataset
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers.composition import ElementProperty, Stoichiometry, ValenceOrbital, IonProperty
from matminer.featurizers.structure import (SiteStatsFingerprint, StructuralHeterogeneity,
                                            ChemicalOrdering, StructureComposition, MaximumPackingEfficiency)
from matminer.featurizers.conversions import DictToObject
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from scipy import stats
from tqdm import tqdm_notebook as tqdm
import numpy as np

## 创建特征

In [2]:
featurizer = MultipleFeaturizer([
    SiteStatsFingerprint.from_preset("CoordinationNumber_ward-prb-2017"),
    StructuralHeterogeneity(),
    ChemicalOrdering(),
    MaximumPackingEfficiency(),
    SiteStatsFingerprint.from_preset("LocalPropertyDifference_ward-prb-2017"),
    StructureComposition(Stoichiometry()),
    StructureComposition(ElementProperty.from_preset("magpie")),
    StructureComposition(ValenceOrbital(props=['frac'])),
    StructureComposition(IonProperty(fast=True))
])

## 加载数据集

In [4]:
%%time
data = load_dataset("flla")
print("Loaded {} entries".format(len(data)))

Fetching flla.json.gz from https://ndownloader.figshare.com/files/13220597 to C:\Users\zefengli\AppData\Roaming\Python\Python39\site-packages\matminer\datasets\flla.json.gz


Fetching https://ndownloader.figshare.com/files/13220597 in MB: 2.607104MB [00:00, 372.13MB/s]                          


Loaded 3938 entries
CPU times: total: 2.56 s
Wall time: 7.63 s


In [5]:
dto = DictToObject(target_col_id="structure", overwrite_data=True)
data = dto.featurize_dataframe(data, "structure")

DictToObject:   0%|          | 0/3938 [00:00<?, ?it/s]

In [6]:
%%time
print('Total number of features:', len(featurizer.featurize(data['structure'][0])))
print('Number of sites in structure:', len(data['structure'][0]))

Total number of features: 273
Number of sites in structure: 2
CPU times: total: 93.8 ms
Wall time: 108 ms


## 特征化整个数据集

In [10]:
%%time
X = featurizer.featurize_many(data['structure'], ignore_errors=True)

MultipleFeaturizer:   0%|          | 0/3938 [00:00<?, ?it/s]

CPU times: total: 3.06 s
Wall time: 12min 40s


In [11]:
X = np.array(X)
print('Input data shape:', X.shape)

Input data shape: (3938, 273)


In [12]:
import pandas as pd
failed = np.any(pd.isnull(X), axis=1)
print('Number of failed structures: {}/{}'.format(np.sum(failed), len(failed)))

Number of failed structures: 150/3938


In [8]:
data.head()

,material_id,e_above_hull,formula,nsites,formation_energy,formation_energy_per_atom,structure
0,mp-10,0.107405,{'As': 1.0},2,0.214810,0.107405,"[[1.11758409 0.79025129 1.93571242] As, [3.352..."
1,mp-10000,0.000000,"{'S': 1.0, 'Hf': 2.0}",6,-7.520110,-1.253352,"[[0. 0. 8.84051822] S, [0. ..."
2,mp-10004,0.000000,"{'P': 1.0, 'Mo': 3.0}",16,-5.429887,-0.339368,"[[1.62101142 2.03208456 2.71687137] P, [2.9549..."
3,mp-10010,0.000000,"{'Al': 1.0, 'Si': 2.0, 'Co': 2.0}",5,-2.684175,-0.536835,"[[0. 0. 0.] Al, [-2.29850896e-08 2.26918935e+..."
4,mp-10021,0.004781,{'Ga': 1.0},2,0.009563,0.004781,"[[2.39728273 0.77565449 2.62420188] Ga, [0.393..."


# 训练模型

In [13]:
model = Pipeline([
    ('imputer', SimpleImputer()),       # SimpleImputer是处理缺失值的类，这里用均值填充（默认），对数据原地修改
    ('model', RandomForestRegressor(n_estimators=150, n_jobs=-1))
])

In [16]:
%%time
model.fit(X, data['formation_energy_per_atom'])

CPU times: total: 59.4 s
Wall time: 6.38 s


Pipeline(steps=[('imputer', SimpleImputer()),
                ('model', RandomForestRegressor(n_estimators=150, n_jobs=-1))])

In [18]:
maes = []
for train_ids, test_ids in tqdm(ShuffleSplit(train_size=3000, n_splits=20).split(X)):           # ShuffleSplit是随机划分数据集的类，这里是随机划分训练集和测试集; n_splits是划分的次数
    train_X = X[train_ids, :]                                                                   # tqdm是进度条的类
    train_y = data['formation_energy_per_atom'].iloc[train_ids]
    test_X = X[test_ids, :]
    test_y = data['formation_energy_per_atom'].iloc[test_ids]

    # 训练模型
    model.fit(train_X, train_y)

    # 预测
    predict_y = model.predict(test_X)
    maes.append(np.abs(test_y - predict_y).mean())

C:\Users\zefengli\AppData\Local\Temp\ipykernel_58700\3215821010.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for train_ids, test_ids in tqdm(ShuffleSplit(train_size=3000, n_splits=20).split(X)):


0it [00:00, ?it/s]

In [20]:
print('MAE: {:.3f}+/-{:.3f} eV/atom'.format(np.mean(maes), stats.sem(maes)))

MAE: 0.176+/-0.002 eV/atom


# 保存模型

In [22]:
import joblib

joblib.dump(model, 'rf.joblib.dat')

['rf.joblib.dat']

In [23]:
# 加载模型
loaded_model = joblib.load('rf.joblib.dat')

In [ ]:
# 预测
